In [1]:
import os
import io
import pandas as pd
pd.set_option("display.max_columns", None)
import geopandas as gp
import boto3
import maup
maup.progress.enabled = True
from s3_paths import *
from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)


#Function to adjust Census VAP to account for incarcerated pop by subtracting and then assign 0 to any negative values
def modify_P0040001_P0050003(block_gdf):
    block_gdf['P0040001-P0050003'] = block_gdf['P0040001'] - block_gdf['P0050003']
    print("Error raised iff the number of incarcerated blocks with non-zero population does NOT equal the number of modified vap blocks.")
    assert (len(block_gdf[block_gdf['P0050003']!=0]) - len(block_gdf[block_gdf['P0040001']!=block_gdf['P0040001-P0050003']])) == 0
    block_gdf.loc[block_gdf['P0040001-P0050003']<0, 'P0040001-P0050003'] = 0

    return block_gdf


#Functions for running maup
def check_valid_rows(block_gdf, precinct_gdf):
    prec_geom = precinct_gdf.geometry
    valid_rows = precinct_gdf[~(prec_geom.isna() | prec_geom.is_empty)]
    print('valid precinct rows: ', valid_rows.shape)

    block_geom = block_gdf.geometry
    block_valid_rows = block_gdf[~(block_geom.isna() | block_geom.is_empty)]
    print('valid block rows: ', block_valid_rows.shape)
    

def fix_buffer(gdf):
    """
    return (GeoDataFrame) with the 'buffer(0) trick' applied
    :gdf: (GeoDataFrame) object
    Can be useful when trying to mitigate 'self-intersection' issues
    """
    buffered = gdf.buffer(0)
    gdf.drop(columns=["geometry"])
    # gdf['geometry'] = gdf.apply(lambda x: x.geometry.buffer(0), axis=1)
    gdf["geometry"] = buffered
    return gdf


def maup_assignment_series(block_gdf, precinct_gdf):
    block_gdf = block_gdf.to_crs(precinct_gdf.crs)
    print('block_gdf shape: ', block_gdf.shape, '\n precinct_gdf shape: ', precinct_gdf.shape)

    block_gdf["maup_assignment"] = maup.assign(
        fix_buffer(block_gdf), fix_buffer(precinct_gdf)).fillna(0) #In case of OH, na are 0 pop
    print(block_gdf[block_gdf['maup_assignment'].isna()].shape[0], " null assignments in maup series")
    return block_gdf


def maup_pre_vote_setup(block_gdf, precinct_gdf, PRECID_block):
    block_gdf = block_gdf.to_crs(precinct_gdf.crs)
    bgdf = maup_assignment_series(block_gdf, precinct_gdf)
    #assert bgdf['maup_assignment'].isna().any() == False  #-- Cut out for OH because all but two are 0 pop
    bgdf[PRECID_block] = bgdf['maup_assignment'].map(lambda idx: str(precinct_gdf.loc[idx]["UNIQUE_ID"]))
    
    return bgdf

    
def maup_assign_labels(block_gdf, precinct_gdf, precinctid_input, distid_input):
    #Assign precinct IDs to block file using assign series
    block_gdf[precinctid_input] = block_gdf["maup_assignment"].map(
        lambda idx: str(precinct_gdf.loc[idx]["UNIQUE_ID"])
    )
    if "CONG_DIST" in precinct_gdf.columns:
        block_gdf[distid_input] = block_gdf["maup_assignment"].map(
            lambda idx: str(precinct_gdf.loc[idx]["CONG_DIST"])
        )
    if "SLDL_DIST" in precinct_gdf.columns:
        block_gdf[distid_input] = block_gdf["maup_assignment"].map(
            lambda idx: str(precinct_gdf.loc[idx]["SLDL_DIST"])
        )
    if "SLDU_DIST" in precinct_gdf.columns:
        block_gdf[distid_input] = block_gdf["maup_assignment"].map(
            lambda idx: str(precinct_gdf.loc[idx]["SLDU_DIST"])
        )
    print("Label assignment complete")

    return block_gdf


def election_cols(gdf):
    election_cols = list(gdf.columns[(gdf.columns.str.startswith('G'))&(~gdf.columns.str.startswith("GEO"))])
    return election_cols 


def assign_votes(variables, election_columns, precinct_gdf, block_gdf):
    precinct_gdf[variables] = block_gdf[variables].groupby(block_gdf["maup_assignment"]).sum()
    print(variables, ' added to precinct_gdf based on maup assignment')
    bl_to_prec_weights = block_gdf[variables] / block_gdf["maup_assignment"].map(precinct_gdf[variables])
    block_votes = block_gdf.copy(deep=True)
    block_votes[election_columns] = maup.prorate(
        block_gdf["maup_assignment"], precinct_gdf[election_columns], bl_to_prec_weights
    )
    return block_votes


def run_maup(block_gdf, precinct_gdf, precinctid_input, pop_variable):
    #Assign identifiers
    block_gdf = maup_assignment_series(block_gdf, precinct_gdf)
    block_gdf = maup_assign_labels(block_gdf, precinct_gdf, precinctid_input)

    #Assign votes
    election_columns = election_cols(precinct_gdf)
    block_votes = assign_votes(pop_variable, election_columns, precinct_gdf, block_gdf)
    block_votes["VAP_MOD"] = block_votes['P0040001-P0050003']
    block_votes["PRECINCTID"] = block_votes["PREC"]
    
    return block_votes   


def column_total_check(election_columns, block_gdf, precinct_gdf):
    mismatch_list = []
    for val in election_columns:
        vote_dif = block_gdf[val].sum()-precinct_gdf[val].sum()
        if (abs(vote_dif) <=1e-1):
            #print(val+": EQUAL", ' - total: ', 'block:', str(block_gdf[val].sum()), 'prec:', str(precinct_gdf[val].sum()), 'diff:', block_gdf[val].sum()-precinct_gdf[val].sum())
            continue
        else:
            mismatch_list.append(val)
            print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES", ' - block total: ', str(block_gdf[val].sum()), ', precinct total: ', str(precinct_gdf[val].sum()))  
    print("Mismatch list: ", mismatch_list)
            
def precinct_sum_check(prec_gdf, block_gdf, blk_prec_id):
    prec_gpbyprec = prec_gdf.groupby(['UNIQUE_ID']).sum()
    blk_gpbyprec = block_gdf.groupby([blk_prec_id]).sum()
    prec_w_difs_list = []
    
    prec_gpbyprec.sort_values(by='UNIQUE_ID',inplace=True)
    prec_gpbyprec.reset_index(inplace = True)
    blk_gpbyprec.sort_values(by= blk_prec_id, inplace=True)
    blk_gpbyprec.reset_index(inplace = True)
    
    for i in range(0, prec_gdf['UNIQUE_ID'].nunique()):
        for col in election_cols(prec_gdf):
            prec_diff = round(blk_gpbyprec[col].iloc[i], 1) - round(prec_gpbyprec[col].iloc[i], 1)
            if (prec_diff!=0):
                prec_w_difs_list.append(prec_gpbyprec.index[i])
                print("prec id: ", blk_gpbyprec.index[i], prec_gpbyprec.index[i], col, "block: ", round(blk_gpbyprec[col].iloc[i],1), 
                      "prec: ", round(prec_gpbyprec[col].iloc[i],1), "diff: ", prec_diff)
    print("precs to check: ", set(prec_w_difs_list))
    print("Precinct check complete!")



In [2]:
##Step 0: Load in relevant gdfs 
s3_client = boto3.client('s3')
census_block_gdf = gp.read_file(f'zip+{census_block}')[['GEOID20', 'COUNTYFP20', 'NAME20','P0010001', 'P0020001', 'P0040001', 'P0050003', 'geometry']]
census_block_gdf = modify_P0040001_P0050003(census_block_gdf)
oh_blocks = census_block_gdf.copy()

Error raised iff the number of incarcerated blocks with non-zero population does NOT equal the number of modified vap blocks.


In [4]:
#Step 0: Load in relevant gdfs 
pber_st = gp.read_file("../../pber_collection/OH/2022/oh_2022_gen_prec_shp/oh_2022_gen_prec_st.shp")
pber_cong = gp.read_file("../../pber_collection/OH/2022/oh_2022_gen_prec_shp/oh_2022_gen_prec_cong.shp")
pber_sl = gp.read_file("../../pber_collection/OH/2022/oh_2022_gen_prec_shp/oh_2022_gen_prec_sldl.shp")
pber_su = gp.read_file("../../pber_collection/OH/2022/oh_2022_gen_prec_shp/oh_2022_gen_prec_sldu.shp")

In [7]:
st_bprec = maup_pre_vote_setup(oh_blocks, pber_st, "PREC")
st_bprec_votes = assign_votes("P0040001-P0050003", election_cols(pber_st), pber_st, st_bprec)

cong_bprec = maup_pre_vote_setup(oh_blocks, pber_cong, "PREC")
cong_bprec_votes = assign_votes("P0040001-P0050003", election_cols(pber_cong), pber_cong, cong_bprec)

sl_bprec = maup_pre_vote_setup(oh_blocks, pber_sl, "PREC")
sl_bprec_votes = assign_votes("P0040001-P0050003", election_cols(pber_sl), pber_sl, sl_bprec)

su_bprec = maup_pre_vote_setup(oh_blocks, pber_su, "PREC")
su_bprec_votes = assign_votes("P0040001-P0050003", election_cols(pber_su), pber_su, su_bprec)

block_gdf shape:  (276428, 9) 
 precinct_gdf shape:  (8941, 27)


100%|██████████| 8941/8941 [01:50<00:00, 81.19it/s] 
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


0  null assignments in maup series
P0040001-P0050003  added to precinct_gdf based on maup assignment
block_gdf shape:  (276428, 9) 
 precinct_gdf shape:  (8977, 39)


100%|██████████| 8977/8977 [02:35<00:00, 57.78it/s]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


0  null assignments in maup series
P0040001-P0050003  added to precinct_gdf based on maup assignment
block_gdf shape:  (276428, 9) 
 precinct_gdf shape:  (9068, 180)


...

0  null assignments in maup series
P0040001-P0050003  added to precinct_gdf based on maup assignment


/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o

block_gdf shape:  (276428, 9) 
 precinct_gdf shape:  (8970, 38)


100%|██████████| 8970/8970 [02:40<00:00, 55.82it/s]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


0  null assignments in maup series
P0040001-P0050003  added to precinct_gdf based on maup assignment


## More checks

In [26]:
cong_bprec[cong_bprec["maup_assignment"]==1309]

,GEOID20,COUNTYFP20,NAME20,P0010001,P0020001,P0040001,P0050003,geometry,P0040001-P0050003,maup_assignment,PREC
11453,390351959002003,035,Block 2003,53,53,39,0,"POLYGON ((-81.38884 41.43223, -81.38807 41.432...",39,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B
20115,390351959002015,035,Block 2015,4,4,1,0,"POLYGON ((-81.39186 41.43111, -81.39116 41.431...",1,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B
24511,390351959004010,035,Block 4010,324,324,265,0,"POLYGON ((-81.39781 41.44079, -81.39578 41.441...",265,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B
50524,390351959002004,035,Block 2004,10,10,10,0,"POLYGON ((-81.38743 41.43369, -81.38580 41.434...",10,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B
54825,390351959002012,035,Block 2012,100,100,84,0,"POLYGON ((-81.38965 41.42900, -81.38929 41.429...",84,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B
63354,390351959004011,035,Block 4011,21,21,16,0,"POLYGON ((-81.38994 41.43508, -81.38925 41.435...",16,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B
84768,390351959002013,035,Block 2013,15,15,15,0,"POLYGON ((-81.37813 41.42510, -81.37812 41.425...",15,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B
95248,390351959002002,035,Block 2002,101,101,94,0,"POLYGON ((-81.38740 41.43219, -81.38629 41.432...",94,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B
105819,390351959002006,035,Block 2006,43,43,29,0,"POLYGON ((-81.38598 41.43310, -81.38462 41.433...",29,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B
110618,390351959004015,035,Block 4015,36,36,28,0,"POLYGON ((-81.39305 41.43307, -81.38986 41.434...",28,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B


In [29]:
cong_bprec_votes["GCON11DBRO"][cong_bprec_votes["maup_assignment"]==1309].sum()

398.99999999999994

In [10]:
pber_cong["GCON11DBRO"].sum()

167722.0

In [33]:
pber_cong["GCON11DBRO"][pber_cong.index==1308]#["GCON11DBRO"].sum()

1308    0.0
Name: GCON11DBRO, dtype: float64

In [34]:
cong_bprec_votes["GCON11DBRO"][cong_bprec_votes["maup_assignment"]==1308].sum()

0.0

In [13]:
pber_cong["GCON11DBRO"][pber_cong.index==1309]#["GCON11DBRO"].sum()

1309    399.0
Name: GCON11DBRO, dtype: float64

In [31]:
cong_bprec_votes["GCON11RBRE"][cong_bprec_votes["maup_assignment"]==1310].sum()

281.00000000000006

In [127]:
cong_bprec_votes.drop("GEOID20", axis=1)

,COUNTYFP20,NAME20,P0010001,P0020001,P0040001,P0050003,geometry,P0040001-P0050003,maup_assignment,PREC,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR
0,061,Block 1012,0,0,0,0,"POLYGON ((-84.52044 39.10441, -84.51994 39.104...",0,3938.0,HAMILTON-CINCINNATI 16-A,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,061,Block 1026,166,166,119,0,"POLYGON ((-84.43199 39.10851, -84.43144 39.108...",119,3895.0,HAMILTON-CINCINNATI 1-J,42.492958,23.267606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,061,Block 5008,64,64,53,0,"POLYGON ((-84.52291 39.22418, -84.52216 39.224...",53,4332.0,HAMILTON-SPRINGFIELD C,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.996127,8.415957,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,061,Block 3005,38,38,34,0,"POLYGON ((-84.51682 39.21901, -84.51655 39.220...",34,4347.0,HAMILTON-SPRINGFIELD R,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.371966,5.351605,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,061,Block 2005,33,33,24,0,"POLYGON ((-84.46973 39.22981, -84.46959 39.229...",24,4396.0,HAMILTON-WYOMING D,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.635168,4.575185,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276423,031,Block 3025,37,37,26,0,"POLYGON ((-81.75083 40.27047, -81.75082 40.270...",26,1168.0,COSHOCTON-ABI,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.313986,6.539525,0.0,0.0,0.0,0.0,0.0,0.0
276424,031,Block 3027,8,8,3,0,"POLYGON ((-82.00400 40.33839, -82.00354 40.338...",3,1161.0,COSHOCTON-AAW,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.293091,1.216329,0.0,0.0,0.0,0.0,0.0,0.0
276425,031,Block 1015,28,28,27,0,"POLYGON ((-81.64777 40.26734, -81.64762 40.267...",27,1165.0,COSHOCTON-ABF,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.452555,10.313869,0.0,0.0,0.0,0.0,0.0,0.0
276426,031,Block 3078,0,0,0,0,"POLYGON ((-81.89382 40.30224, -81.89357 40.302...",0,1167.0,COSHOCTON-ABH,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
pber_cong["GCON11RBRE"][pber_cong.index==1310]#["GCON11DBRO"].sum()

1310    281.0
Name: GCON11RBRE, dtype: float64

In [21]:
pber_cong[pber_cong.index==1310]

,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,PRECCODE,VTDST22,CON_DIST,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR,geometry,P0040001-P0050003
1310,CUYAHOGA-CHAGRIN FALLS-00-C,035,CUYAHOGA,CHAGRIN FALLS,DAB,035DAB,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,295.0,281.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-81.37600 41.42778, -81.37750 41.428...",1016.0


In [17]:
pber_cong[pber_cong.index==1309]

,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,PRECCODE,VTDST22,CON_DIST,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR,geometry,P0040001-P0050003
1309,CUYAHOGA-CHAGRIN FALLS-00-B,035,CUYAHOGA,CHAGRIN FALLS,DAA,035DAA,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,399.0,348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-81.39889 41.44248, -81.39889 41.442...",1003.0


In [22]:
cong_bprec_votes

,GEOID20,COUNTYFP20,NAME20,P0010001,P0020001,P0040001,P0050003,geometry,P0040001-P0050003,maup_assignment,PREC,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR
0,390610265001012,061,Block 1012,0,0,0,0,"POLYGON ((-84.52044 39.10441, -84.51994 39.104...",0,3938.0,HAMILTON-CINCINNATI 16-A,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,390610047031026,061,Block 1026,166,166,119,0,"POLYGON ((-84.43199 39.10851, -84.43144 39.108...",119,3895.0,HAMILTON-CINCINNATI 1-J,42.492958,23.267606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,390610221025008,061,Block 5008,64,64,53,0,"POLYGON ((-84.52291 39.22418, -84.52216 39.224...",53,4332.0,HAMILTON-SPRINGFIELD C,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.996127,8.415957,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,390610221023005,061,Block 3005,38,38,34,0,"POLYGON ((-84.51682 39.21901, -84.51655 39.220...",34,4347.0,HAMILTON-SPRINGFIELD R,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.371966,5.351605,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,390610226022005,061,Block 2005,33,33,24,0,"POLYGON ((-84.46973 39.22981, -84.46959 39.229...",24,4396.0,HAMILTON-WYOMING D,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.635168,4.575185,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276423,390319618003025,031,Block 3025,37,37,26,0,"POLYGON ((-81.75083 40.27047, -81.75082 40.270...",26,1168.0,COSHOCTON-ABI,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.313986,6.539525,0.0,0.0,0.0,0.0,0.0,0.0
276424,390319610003027,031,Block 3027,8,8,3,0,"POLYGON ((-82.00400 40.33839, -82.00354 40.338...",3,1161.0,COSHOCTON-AAW,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.293091,1.216329,0.0,0.0,0.0,0.0,0.0,0.0
276425,390319618001015,031,Block 1015,28,28,27,0,"POLYGON ((-81.64777 40.26734, -81.64762 40.267...",27,1165.0,COSHOCTON-ABF,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.452555,10.313869,0.0,0.0,0.0,0.0,0.0,0.0
276426,390319613003078,031,Block 3078,0,0,0,0,"POLYGON ((-81.89382 40.30224, -81.89357 40.302...",0,1167.0,COSHOCTON-ABH,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


prec id:  1309 1309 GCON11DBRO block:  295.0 prec:  399.0 diff:  -104.0

prec id:  1309 1309 GCON11RBRE block:  281.0 prec:  348.0 diff:  -67.0

In [37]:
cong_bprec_votes["GCON11DBRO"][cong_bprec_votes["PREC"]=="CUYAHOGA-CHAGRIN FALLS-00-B"].sum()

398.99999999999994

In [40]:
pber_cong["GCON11DBRO"][pber_cong["UNIQUE_ID"]=="CUYAHOGA-CHAGRIN FALLS-00-B"].sum()

399.0

In [60]:
pber_cong["GCON11DBRO"][pber_cong["UNIQUE_ID"]=="CUYAHOGA-CHAGRIN FALLS-00-B"]

1309    399.0
Name: GCON11DBRO, dtype: float64

In [62]:
cong_bprec_votes[cong_bprec_votes["PREC"]=="CUYAHOGA-CHAGRIN FALLS-00-B"]

,GEOID20,COUNTYFP20,NAME20,P0010001,P0020001,P0040001,P0050003,geometry,P0040001-P0050003,maup_assignment,PREC,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR
11453,390351959002003,035,Block 2003,53,53,39,0,"POLYGON ((-81.38884 41.43223, -81.38807 41.432...",39,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.514457,13.531406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20115,390351959002015,035,Block 2015,4,4,1,0,"POLYGON ((-81.39186 41.43111, -81.39116 41.431...",1,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.397807,0.346959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24511,390351959004010,035,Block 4010,324,324,265,0,"POLYGON ((-81.39781 41.44079, -81.39578 41.441...",265,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105.418744,91.944167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50524,390351959002004,035,Block 2004,10,10,10,0,"POLYGON ((-81.38743 41.43369, -81.38580 41.434...",10,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.978066,3.469591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54825,390351959002012,035,Block 2012,100,100,84,0,"POLYGON ((-81.38965 41.42900, -81.38929 41.429...",84,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.415753,29.144566,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63354,390351959004011,035,Block 4011,21,21,16,0,"POLYGON ((-81.38994 41.43508, -81.38925 41.435...",16,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.364905,5.551346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84768,390351959002013,035,Block 2013,15,15,15,0,"POLYGON ((-81.37813 41.42510, -81.37812 41.425...",15,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.967099,5.204387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95248,390351959002002,035,Block 2002,101,101,94,0,"POLYGON ((-81.38740 41.43219, -81.38629 41.432...",94,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.393819,32.614158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105819,390351959002006,035,Block 2006,43,43,29,0,"POLYGON ((-81.38598 41.43310, -81.38462 41.433...",29,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.536391,10.061815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110618,390351959004015,035,Block 4015,36,36,28,0,"POLYGON ((-81.39305 41.43307, -81.38986 41.434...",28,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.138584,9.714855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
cong_bprec_votes[cong_bprec_votes["PREC"]=="CUYAHOGA-CHAGRIN FALLS-00-B"]

,GEOID20,COUNTYFP20,NAME20,P0010001,P0020001,P0040001,P0050003,geometry,P0040001-P0050003,maup_assignment,PREC,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR
11453,390351959002003,035,Block 2003,53,53,39,0,"POLYGON ((-81.38884 41.43223, -81.38807 41.432...",39,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.514457,13.531406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20115,390351959002015,035,Block 2015,4,4,1,0,"POLYGON ((-81.39186 41.43111, -81.39116 41.431...",1,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.397807,0.346959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24511,390351959004010,035,Block 4010,324,324,265,0,"POLYGON ((-81.39781 41.44079, -81.39578 41.441...",265,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105.418744,91.944167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50524,390351959002004,035,Block 2004,10,10,10,0,"POLYGON ((-81.38743 41.43369, -81.38580 41.434...",10,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.978066,3.469591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54825,390351959002012,035,Block 2012,100,100,84,0,"POLYGON ((-81.38965 41.42900, -81.38929 41.429...",84,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.415753,29.144566,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63354,390351959004011,035,Block 4011,21,21,16,0,"POLYGON ((-81.38994 41.43508, -81.38925 41.435...",16,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.364905,5.551346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84768,390351959002013,035,Block 2013,15,15,15,0,"POLYGON ((-81.37813 41.42510, -81.37812 41.425...",15,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.967099,5.204387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95248,390351959002002,035,Block 2002,101,101,94,0,"POLYGON ((-81.38740 41.43219, -81.38629 41.432...",94,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.393819,32.614158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105819,390351959002006,035,Block 2006,43,43,29,0,"POLYGON ((-81.38598 41.43310, -81.38462 41.433...",29,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.536391,10.061815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110618,390351959004015,035,Block 4015,36,36,28,0,"POLYGON ((-81.39305 41.43307, -81.38986 41.434...",28,1309.0,CUYAHOGA-CHAGRIN FALLS-00-B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.138584,9.714855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
cong_bprec_votes

,GEOID20,COUNTYFP20,NAME20,P0010001,P0020001,P0040001,P0050003,geometry,P0040001-P0050003,maup_assignment,PREC,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR
0,390610265001012,061,Block 1012,0,0,0,0,"POLYGON ((-84.52044 39.10441, -84.51994 39.104...",0,3938.0,HAMILTON-CINCINNATI 16-A,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,390610047031026,061,Block 1026,166,166,119,0,"POLYGON ((-84.43199 39.10851, -84.43144 39.108...",119,3895.0,HAMILTON-CINCINNATI 1-J,42.492958,23.267606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,390610221025008,061,Block 5008,64,64,53,0,"POLYGON ((-84.52291 39.22418, -84.52216 39.224...",53,4332.0,HAMILTON-SPRINGFIELD C,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.996127,8.415957,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,390610221023005,061,Block 3005,38,38,34,0,"POLYGON ((-84.51682 39.21901, -84.51655 39.220...",34,4347.0,HAMILTON-SPRINGFIELD R,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.371966,5.351605,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,390610226022005,061,Block 2005,33,33,24,0,"POLYGON ((-84.46973 39.22981, -84.46959 39.229...",24,4396.0,HAMILTON-WYOMING D,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.635168,4.575185,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276423,390319618003025,031,Block 3025,37,37,26,0,"POLYGON ((-81.75083 40.27047, -81.75082 40.270...",26,1168.0,COSHOCTON-ABI,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.313986,6.539525,0.0,0.0,0.0,0.0,0.0,0.0
276424,390319610003027,031,Block 3027,8,8,3,0,"POLYGON ((-82.00400 40.33839, -82.00354 40.338...",3,1161.0,COSHOCTON-AAW,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.293091,1.216329,0.0,0.0,0.0,0.0,0.0,0.0
276425,390319618001015,031,Block 1015,28,28,27,0,"POLYGON ((-81.64777 40.26734, -81.64762 40.267...",27,1165.0,COSHOCTON-ABF,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.452555,10.313869,0.0,0.0,0.0,0.0,0.0,0.0
276426,390319613003078,031,Block 3078,0,0,0,0,"POLYGON ((-81.89382 40.30224, -81.89357 40.302...",0,1167.0,COSHOCTON-ABH,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


## BLOCK PREC ID
- use index or use PREC which is prec name? Are both failing or just one?
- Individual checks seem to be in favor of block and prec matching, why is the fxn failing?

In [41]:
pber_cong[pber_cong.columns[pber_cong.columns!="geometry"]]

,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,PRECCODE,VTDST22,CON_DIST,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR,P0040001-P0050003
0,ADAMS-AAA,001,ADAMS,AAA,AAA,001AAA,02,0.0,0.0,80.0,489.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1062.0
1,ADAMS-AAB,001,ADAMS,AAB,AAB,001AAB,02,0.0,0.0,61.0,301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,862.0
2,ADAMS-AAD,001,ADAMS,AAD,AAD,001AAD,02,0.0,0.0,54.0,365.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,867.0
3,ADAMS-AAE,001,ADAMS,AAE,AAE,001AAE,02,0.0,0.0,36.0,154.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,409.0
4,ADAMS-AAG,001,ADAMS,AAG,AAG,001AAG,02,0.0,0.0,42.0,201.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,724.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8972,WYANDOT-ABF,175,WYANDOT,ABF,ABF,175ABF,05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,637.0
8973,WYANDOT-ABH,175,WYANDOT,ABH,ABH,175ABH,05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,387.0
8974,WYANDOT-ABI,175,WYANDOT,ABI,ABI,175ABI,05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.0,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,793.0
8975,WYANDOT-ABL,175,WYANDOT,ABL,ABL,175ABL,05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,446.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1131.0


In [44]:
pber_st[pber_st.columns[pber_st.columns!="geometry"]].reset_index()

,index,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,PRECCODE,VTDST22,G22ATGDCRO,G22ATGRYOS,G22AUDDSAP,G22AUDRFAB,G22CJUDBRU,G22CJURKEN,G22GOVDWHA,G22GOVRDEW,G22JUSDJAM,G22JUSDZAY,G22JUSRDEW,G22JUSRFIS,G22SOSDCLA,G22SOSOMAR,G22SOSRLAR,G22TREDSCH,G22TRERSPR,G22USSDRYA,G22USSRVAN,P0040001-P0050003
0,0,ADAMS-AAA,001,ADAMS,AAA,AAA,001AAA,81.0,485.0,91.0,469.0,94.0,469.0,70.0,491.0,92.0,96.0,470.0,472.0,83.0,8.0,474.0,85.0,478.0,106.0,454.0,1062
1,1,ADAMS-AAB,001,ADAMS,AAB,AAB,001AAB,64.0,292.0,67.0,284.0,64.0,291.0,49.0,306.0,67.0,70.0,289.0,286.0,58.0,4.0,295.0,66.0,288.0,78.0,276.0,862
2,2,ADAMS-AAD,001,ADAMS,AAD,AAD,001AAD,54.0,359.0,51.0,360.0,61.0,353.0,48.0,366.0,58.0,59.0,350.0,353.0,53.0,2.0,364.0,58.0,354.0,70.0,337.0,867
3,3,ADAMS-AAE,001,ADAMS,AAE,AAE,001AAE,35.0,152.0,39.0,143.0,40.0,146.0,38.0,148.0,42.0,40.0,146.0,144.0,38.0,3.0,145.0,36.0,148.0,45.0,140.0,409
4,4,ADAMS-AAG,001,ADAMS,AAG,AAG,001AAG,45.0,192.0,45.0,192.0,47.0,189.0,44.0,191.0,48.0,48.0,190.0,185.0,45.0,2.0,190.0,48.0,190.0,53.0,182.0,724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8936,8936,WYANDOT-ABF,175,WYANDOT,ABF,ABF,175ABF,39.0,310.0,41.0,301.0,45.0,303.0,40.0,301.0,42.0,53.0,292.0,301.0,42.0,3.0,301.0,40.0,305.0,57.0,282.0,637
8937,8937,WYANDOT-ABH,175,WYANDOT,ABH,ABH,175ABH,37.0,179.0,36.0,178.0,40.0,176.0,34.0,180.0,39.0,44.0,170.0,175.0,33.0,4.0,177.0,36.0,179.0,50.0,167.0,387
8938,8938,WYANDOT-ABI,175,WYANDOT,ABI,ABI,175ABI,73.0,347.0,76.0,337.0,86.0,333.0,73.0,340.0,86.0,97.0,320.0,330.0,72.0,7.0,337.0,87.0,332.0,107.0,308.0,793
8939,8939,WYANDOT-ABL,175,WYANDOT,ABL,ABL,175ABL,111.0,452.0,115.0,449.0,119.0,445.0,100.0,464.0,117.0,127.0,433.0,448.0,103.0,4.0,458.0,112.0,450.0,152.0,405.0,1131


In [65]:
cong_gp_by_blk = cong_bprec_votes[cong_bprec_votes.columns[cong_bprec_votes.columns!="geometry"]].groupby("maup_assignment").sum()
cong_gp_by_blk.shape

(8974, 39)

In [66]:
pber_cong.shape

(8977, 39)

In [51]:
pber_all = gp.read_file("../../pber_collection/OH/2022/oh_2022_gen_prec_shp/oh_2022_gen_prec_no_splits.shp")

In [52]:
pber_all["GCON01DLAN"].sum()

156416.0

In [46]:
pber_cong["GCON01DLAN"].sum()

156416.0

In [47]:
cong_bprec_votes["GCON01DLAN"].sum()

156368.00000000003

In [48]:
precinct_sum_check(pber_su[pber_su.columns[pber_su.columns!="geometry"]], su_bprec_votes[su_bprec_votes.columns[su_bprec_votes.columns!="geometry"]], "PREC")

prec id:  2672 2672 GSU15DCRA block:  0.0 prec:  1.0 diff:  -1.0
prec id:  2672 2672 GSU25DDEM block:  261.0 prec:  0.0 diff:  261.0
prec id:  2672 2672 GSU25RWYS block:  79.0 prec:  0.0 diff:  79.0
prec id:  2673 2673 GSU15DCRA block:  1.0 prec:  0.0 diff:  1.0
prec id:  2673 2673 GSU25DDEM block:  0.0 prec:  261.0 diff:  -261.0
prec id:  2673 2673 GSU25RWYS block:  0.0 prec:  79.0 diff:  -79.0
prec id:  2674 2674 GSU15DCRA block:  0.0 prec:  1.0 diff:  -1.0
prec id:  2674 2674 GSU25DDEM block:  393.0 prec:  0.0 diff:  393.0
prec id:  2674 2674 GSU25RWYS block:  147.0 prec:  0.0 diff:  147.0
prec id:  2675 2675 GSU25DDEM block:  465.0 prec:  393.0 diff:  72.0
prec id:  2675 2675 GSU25RWYS block:  274.0 prec:  147.0 diff:  127.0
prec id:  2676 2676 GSU03DMAH block:  92.0 prec:  0.0 diff:  92.0
prec id:  2676 2676 GSU03RREY block:  58.0 prec:  0.0 diff:  58.0
prec id:  2676 2676 GSU25DDEM block:  0.0 prec:  465.0 diff:  -465.0
prec id:  2676 2676 GSU25RWYS block:  0.0 prec:  274.0 diff:

IndexError: single positional indexer is out-of-bounds

In [54]:
set(pber_cong["UNIQUE_ID"])- set(cong_bprec_votes["PREC"])

{'CUYAHOGA-CHAGRIN FALLS-00-A-(CON-14)',
 'DELAWARE-PRECINCT GENOA L-(CON-04)',
 'DELAWARE-PRECINCT ORANGE T-(CON-12)'}

In [69]:
os.mkdir("./pber_cong_missing_prec/")
pber_cong[pber_cong["UNIQUE_ID"].isin({'CUYAHOGA-CHAGRIN FALLS-00-A-(CON-14)','DELAWARE-PRECINCT GENOA L-(CON-04)','DELAWARE-PRECINCT ORANGE T-(CON-12)'})].to_file("./pber_cong_missing_prec/pber_cong_missing_prec.shp")

<ipython-input-69-c75c8b5f8f0c>:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  pber_cong[pber_cong["UNIQUE_ID"].isin({'CUYAHOGA-CHAGRIN FALLS-00-A-(CON-14)','DELAWARE-PRECINCT GENOA L-(CON-04)','DELAWARE-PRECINCT ORANGE T-(CON-12)'})].to_file("./pber_cong_missing_prec/pber_cong_missing_prec.shp")


In [55]:
set(cong_bprec_votes["PREC"])-set(pber_cong["UNIQUE_ID"])

set()

In [123]:
len(set(pber_sl["UNIQUE_ID"])- set(sl_bprec_votes["PREC"]))

38

In [122]:
pber_sl[pber_sl["UNIQUE_ID"].isin(set(pber_sl["UNIQUE_ID"])- set(sl_bprec_votes["PREC"]))].to_file("./pber_missing_prec_blk/pber_missing_prec_sl.shp")
pber_su[pber_su["UNIQUE_ID"].isin(set(pber_su["UNIQUE_ID"])- set(su_bprec_votes["PREC"]))].to_file("./pber_missing_prec_blk/pber_missing_prec_su.shp")

<ipython-input-122-2e86a3d8b728>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  pber_sl[pber_sl["UNIQUE_ID"].isin(set(pber_sl["UNIQUE_ID"])- set(sl_bprec_votes["PREC"]))].to_file("./pber_missing_prec_blk/pber_missing_prec_sl.shp")
<ipython-input-122-2e86a3d8b728>:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  pber_su[pber_su["UNIQUE_ID"].isin(set(pber_su["UNIQUE_ID"])- set(su_bprec_votes["PREC"]))].to_file("./pber_missing_prec_blk/pber_missing_prec_su.shp")


In [119]:
column_total_check(election_cols(pber_sl), sl_bprec_votes, pber_sl[~pber_sl["UNIQUE_ID"].isin(set(pber_sl["UNIQUE_ID"])- set(sl_bprec_votes["PREC"]))])

GSL006DMIL: DIFFERENCE OF -2.000000000003638 VOTES  - block total:  15428.999999999996 , precinct total:  15431.0
GSL006RWHA: DIFFERENCE OF -1.000000000001819 VOTES  - block total:  11270.999999999998 , precinct total:  11272.0
Mismatch list:  ['GSL006DMIL', 'GSL006RWHA']


In [120]:
set(pber_su["UNIQUE_ID"])- set(su_bprec_votes["PREC"])

{'FRANKLIN-ACB-(SU-15)',
 'FRANKLIN-BAJ-(SU-03)',
 'FRANKLIN-BBH-(SU-16)',
 'FRANKLIN-BBJ-(SU-16)',
 'FRANKLIN-BBP-(SU-16)',
 'FRANKLIN-BBR-(SU-19)',
 'FRANKLIN-BIQ-(SU-03)',
 'FRANKLIN-BLY-(SU-20)'}

In [121]:
column_total_check(election_cols(pber_su), su_bprec_votes, pber_su[~pber_su["UNIQUE_ID"].isin(set(pber_su["UNIQUE_ID"])- set(su_bprec_votes["PREC"]))])

GSU15DCRA: DIFFERENCE OF -2.0 VOTES  - block total:  71997.0 , precinct total:  71999.0
Mismatch list:  ['GSU15DCRA']


In [57]:
set(sl_bprec_votes["PREC"])-set(pber_sl["UNIQUE_ID"])

set()

In [59]:
column_total_check(election_cols(pber_cong), cong_bprec_votes, pber_cong[~pber_cong["UNIQUE_ID"].isin(set(pber_cong["UNIQUE_ID"])- set(cong_bprec_votes["PREC"]))])

GCON01DLAN: DIFFERENCE OF -47.999999999970896 VOTES  - block total:  156368.00000000003 , precinct total:  156416.0
GCON01RCHA: DIFFERENCE OF -52.0 VOTES  - block total:  140006.0 , precinct total:  140058.0
Mismatch list:  ['GCON01DLAN', 'GCON01RCHA']


In [80]:
pd.Series(cong_bprec_votes["PREC"].values, index =cong_bprec_votes["maup_assignment"].astype(int)).to_dict()

{3938: 'HAMILTON-CINCINNATI 16-A',
 3895: 'HAMILTON-CINCINNATI 1-J',
 4332: 'HAMILTON-SPRINGFIELD C',
 4347: 'HAMILTON-SPRINGFIELD R',
 4396: 'HAMILTON-WYOMING D',
 4003: 'HAMILTON-CINCINNATI 25-E',
 3905: 'HAMILTON-CINCINNATI 11-D',
 8735: 'WASHINGTON-ABR',
 4378: 'HAMILTON-SYMMES E',
 4329: 'HAMILTON-SPRINGFIELD AA',
 3889: 'HAMILTON-CINCINNATI 1-D',
 4053: 'HAMILTON-CINCINNATI 6-B',
 3930: 'HAMILTON-CINCINNATI 15-C',
 4292: 'HAMILTON-NORWOOD 2-C',
 4056: 'HAMILTON-CINCINNATI 6-E',
 4055: 'HAMILTON-CINCINNATI 6-D',
 4356: 'HAMILTON-ST BERNARD 1-A',
 4122: 'HAMILTON-COLUMBIA G',
 4187: 'HAMILTON-GREEN HH',
 4392: 'HAMILTON-WOODLAWN B',
 3946: 'HAMILTON-CINCINNATI 19-C',
 4052: 'HAMILTON-CINCINNATI 6-A',
 4054: 'HAMILTON-CINCINNATI 6-C',
 3927: 'HAMILTON-CINCINNATI 14-I',
 3928: 'HAMILTON-CINCINNATI 15-A',
 4389: 'HAMILTON-WHITEWATER B',
 3886: 'HAMILTON-CINCINNATI 1-A',
 4276: 'HAMILTON-MT HEALTHY 4-A',
 3940: 'HAMILTON-CINCINNATI 17-B',
 3979: 'HAMILTON-CINCINNATI 23-H',
 3957: 'HAMI

In [77]:
pd.Series(pber_cong["UNIQUE_ID"].values, index =pber_cong.index).to_dict()

{0: 'ADAMS-AAA',
 1: 'ADAMS-AAB',
 2: 'ADAMS-AAD',
 3: 'ADAMS-AAE',
 4: 'ADAMS-AAG',
 5: 'ADAMS-AAI',
 6: 'ADAMS-AAK',
 7: 'ADAMS-AAN',
 8: 'ADAMS-AAP',
 9: 'ADAMS-AAR',
 10: 'ADAMS-AAS',
 11: 'ADAMS-AAT',
 12: 'ADAMS-AAU',
 13: 'ADAMS-AAW',
 14: 'ADAMS-AAY',
 15: 'ADAMS-AAZ',
 16: 'ADAMS-ABA',
 17: 'ADAMS-ABC',
 18: 'ADAMS-ABE',
 19: 'ADAMS-ABG',
 20: 'ADAMS-ABH',
 21: 'ALLEN-AAA',
 22: 'ALLEN-AAC',
 23: 'ALLEN-AAE',
 24: 'ALLEN-AAG',
 25: 'ALLEN-AAH',
 26: 'ALLEN-AAJ',
 27: 'ALLEN-AAK',
 28: 'ALLEN-AAO',
 29: 'ALLEN-AAP',
 30: 'ALLEN-AAQ',
 31: 'ALLEN-AAW',
 32: 'ALLEN-AAX',
 33: 'ALLEN-AAY',
 34: 'ALLEN-ABE',
 35: 'ALLEN-ABF',
 36: 'ALLEN-ABH',
 37: 'ALLEN-ABI',
 38: 'ALLEN-ABP',
 39: 'ALLEN-ABQ',
 40: 'ALLEN-ABR',
 41: 'ALLEN-ABT',
 42: 'ALLEN-ABW',
 43: 'ALLEN-ABX',
 44: 'ALLEN-ABZ',
 45: 'ALLEN-ACD',
 46: 'ALLEN-ACE',
 47: 'ALLEN-ACG',
 48: 'ALLEN-ACT',
 49: 'ALLEN-ACU',
 50: 'ALLEN-ACW',
 51: 'ALLEN-ACX',
 52: 'ALLEN-ACY',
 53: 'ALLEN-ACZ',
 54: 'ALLEN-ADA',
 55: 'ALLEN-ADB',
 5

In [81]:
pber_cong["UNIQUE_ID"]

0         ADAMS-AAA
1         ADAMS-AAB
2         ADAMS-AAD
3         ADAMS-AAE
4         ADAMS-AAG
           ...     
8972    WYANDOT-ABF
8973    WYANDOT-ABH
8974    WYANDOT-ABI
8975    WYANDOT-ABL
8976    WYANDOT-ABM
Name: UNIQUE_ID, Length: 8977, dtype: object

In [97]:
#def precinct_sum_check(prec_gdf, block_gdf, blk_prec_id):
prec_gdf = pber_cong[(pber_cong.columns[pber_cong.columns!="geometry"])][(~pber_cong["UNIQUE_ID"].isin({'CUYAHOGA-CHAGRIN FALLS-00-A-(CON-14)','DELAWARE-PRECINCT GENOA L-(CON-04)','DELAWARE-PRECINCT ORANGE T-(CON-12)'}))].copy()
block_gdf = cong_bprec_votes[cong_bprec_votes.columns[cong_bprec_votes.columns!="geometry"]].copy()
#prec_gdf = pber_st[pber_st.columns[pber_st.columns!="geometry"]].copy()
#block_gdf = st_bprec_votes[st_bprec_votes.columns[st_bprec_votes.columns!="geometry"]].copy()

blk_prec_id = "PREC"


prec_gpbyprec = prec_gdf.groupby(['UNIQUE_ID']).sum()
blk_gpbyprec = block_gdf.groupby([blk_prec_id]).sum()
prec_w_difs_list = []

prec_gpbyprec.sort_values(by='UNIQUE_ID',inplace=True)
prec_gpbyprec.reset_index(inplace = True)
blk_gpbyprec.sort_values(by= blk_prec_id, inplace=True)
blk_gpbyprec.reset_index(inplace = True)

for i in range(0, prec_gdf['UNIQUE_ID'].nunique()):
    for col in election_cols(prec_gdf):
        prec_diff = round(blk_gpbyprec[col].iloc[i], 1) - round(prec_gpbyprec[col].iloc[i], 1)
        if (prec_diff!=0):
            prec_w_difs_list.append(prec_gpbyprec.index[i])
            print("prec id: ", blk_gpbyprec.index[i], prec_gpbyprec.index[i], col, "block: ", round(blk_gpbyprec[col].iloc[i],1), 
                  "prec: ", round(prec_gpbyprec[col].iloc[i],1), "diff: ", prec_diff)
print("precs to check: ", set(prec_w_difs_list))
print("Precinct check complete!")


prec id:  4309 4309 GCON01DLAN block:  0.0 prec:  48.0 diff:  -48.0
prec id:  4309 4309 GCON01RCHA block:  0.0 prec:  52.0 diff:  -52.0
precs to check:  {4309}
Precinct check complete!


In [118]:
blk_gpbyprec[blk_gpbyprec.index==4309]

,PREC,GEOID20,COUNTYFP20,NAME20,P0010001,P0020001,P0040001,P0050003,P0040001-P0050003,maup_assignment,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR
4309,HAMILTON-SHARONVILLE 3-A-(CON-01),390610230011004,061,Block 1004,0,0,0,0,0,4312.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
prec_gpbyprec[prec_gpbyprec.index==4309]

,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,PRECCODE,VTDST22,CON_DIST,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR,P0040001-P0050003
4309,HAMILTON-SHARONVILLE 3-A-(CON-01),061,HAMILTON,SHARONVILLE 3,AWZ,061AWZ,01,48.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
pber_cong.shape

(8977, 39)

In [113]:
pber_cong["UNIQUE_ID"].nunique()

8977

In [110]:
prec_gpbyprec["GCON01DLAN"][prec_gpbyprec.index==4309]

4309    48.0
Name: GCON01DLAN, dtype: float64

In [87]:
prec_gpbyprec

,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,PRECCODE,VTDST22,CON_DIST,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR,P0040001-P0050003
0,ADAMS-AAA,001,ADAMS,AAA,AAA,001AAA,02,0.0,0.0,80.0,489.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1062.0
1,ADAMS-AAB,001,ADAMS,AAB,AAB,001AAB,02,0.0,0.0,61.0,301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,862.0
2,ADAMS-AAD,001,ADAMS,AAD,AAD,001AAD,02,0.0,0.0,54.0,365.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,867.0
3,ADAMS-AAE,001,ADAMS,AAE,AAE,001AAE,02,0.0,0.0,36.0,154.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,409.0
4,ADAMS-AAG,001,ADAMS,AAG,AAG,001AAG,02,0.0,0.0,42.0,201.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,724.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8972,WYANDOT-ABF,175,WYANDOT,ABF,ABF,175ABF,05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,637.0
8973,WYANDOT-ABH,175,WYANDOT,ABH,ABH,175ABH,05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,387.0
8974,WYANDOT-ABI,175,WYANDOT,ABI,ABI,175ABI,05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.0,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,793.0
8975,WYANDOT-ABL,175,WYANDOT,ABL,ABL,175ABL,05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,446.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1131.0


In [88]:
blk_gpbyprec

,PREC,GEOID20,COUNTYFP20,NAME20,P0010001,P0020001,P0040001,P0050003,P0040001-P0050003,maup_assignment,GCON01DLAN,GCON01RCHA,GCON02DMEA,GCON02RWEN,GCON03DBEA,GCON03RSTA,GCON04DWIL,GCON04RJOR,GCON05DSWA,GCON05RLAT,GCON06DLYR,GCON06RJOH,GCON07DDIE,GCON07RMIL,GCON08DENO,GCON08RDAV,GCON09DKAP,GCON09RMAJ,GCON10DESR,GCON10RTUR,GCON11DBRO,GCON11RBRE,GCON12DRIP,GCON12RBAL,GCON13DSYK,GCON13RGIL,GCON14DKIL,GCON14RJOY,GCON15DJOS,GCON15RCAR
0,ADAMS-AAA,3900177020010033902995180020093903598010010203...,0010290350010010010010011510350950010010010350...,Block 1003Block 2009Block 1020Block 1014Block ...,1380,1380,1062,0,1062,0.0,0.0,0.0,80.0,489.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ADAMS-AAB,3900177050010493900177050010263900177050030133...,0010010010010010010010010010010010010010010010...,Block 1049Block 1026Block 3013Block 1041Block ...,1097,1097,862,0,862,53.0,0.0,0.0,61.0,301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ADAMS-AAD,3900177010040273900177010010363900177010010443...,0010010010010010010010010010010010010010010010...,Block 4027Block 1036Block 1044Block 3010Block ...,1097,1097,867,0,867,220.0,0.0,0.0,54.0,365.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ADAMS-AAE,3900177050030533900177050030493900177050030193...,0010010010010010010010010010010010010010010010...,Block 3053Block 3049Block 3019Block 3033Block ...,552,552,409,0,409,183.0,0.0,0.0,36.0,154.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ADAMS-AAG,3900177050010633900177050010583900177050010643...,0010010010010010010010010010010010010010010010...,Block 1063Block 1058Block 1064Block 1022Block ...,898,898,724,0,724,212.0,0.0,0.0,42.0,201.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,WYANDOT-ABF,3917593820020303917593820020423917593820020793...,1751751751751751751751751751751751751751751751...,Block 2030Block 2042Block 2079Block 2050Block ...,846,846,637,0,637,708788.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8970,WYANDOT-ABH,3917593820020003917593820010113917593810010423...,1751751751751751751751751751751751751751751751...,Block 2000Block 1011Block 1042Block 1010Block ...,532,532,387,0,387,385839.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8971,WYANDOT-ABI,3917593820010763917593800010863917593820010203...,1751751751751751751751751751751751751751751751...,Block 1076Block 1086Block 1020Block 1084Block ...,981,981,793,0,793,969192.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.0,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8972,WYANDOT-ABL,3917593800040113917593800030493917593800030763...,1751751751751751751751751751751751751751751751...,Block 4011Block 3049Block 3076Block 3083Block ...,1462,1462,1131,0,1131,1023150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,446.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
column_total_check(election_cols(pber_st), st_bprec_votes, pber_st)
column_total_check(election_cols(pber_cong), cong_bprec_votes, pber_cong)
column_total_check(election_cols(pber_sl), sl_bprec_votes, pber_sl)
column_total_check(election_cols(pber_su), su_bprec_votes, pber_su)


precinct_sum_check(pber_st[pber_st.columns[pber_st.columns!="geometry"]], st_bprec_votes[st_bprec_votes.columns[st_bprec_votes.columns!="geometry"]], "PREC")
precinct_sum_check(pber_cong[pber_cong.columns[pber_cong.columns!="geometry"]], cong_bprec_votes[cong_bprec_votes.columns[cong_bprec_votes.columns!="geometry"]], "PREC")
precinct_sum_check(pber_sl[pber_st.columns[pber_sl.columns!="geometry"]], sl_bprec_votes[sl_bprec_votes.columns[sl_bprec_votes.columns!="geometry"]], "PREC")
precinct_sum_check(pber_su[pber_su.columns[pber_su.columns!="geometry"]], su_bprec_votes[su_bprec_votes.columns[su_bprec_votes.columns!="geometry"]], "PREC")

Mismatch list:  []
GCON01DLAN: DIFFERENCE OF -47.999999999970896 VOTES  - block total:  156368.00000000003 , precinct total:  156416.0
GCON01RCHA: DIFFERENCE OF -52.0 VOTES  - block total:  140006.0 , precinct total:  140058.0
GCON04DWIL: DIFFERENCE OF -3.999999999985448 VOTES  - block total:  89379.00000000001 , precinct total:  89383.0
GCON12DRIP: DIFFERENCE OF -1.0000000000291038 VOTES  - block total:  84891.99999999997 , precinct total:  84893.0
GCON12RBAL: DIFFERENCE OF -4.000000000029104 VOTES  - block total:  191339.99999999997 , precinct total:  191344.0
Mismatch list:  ['GCON01DLAN', 'GCON01RCHA', 'GCON04DWIL', 'GCON12DRIP', 'GCON12RBAL']
GSL001DJAR: DIFFERENCE OF -1.0 VOTES  - block total:  31493.0 , precinct total:  31494.0
GSL005RBEA: DIFFERENCE OF -5.999999999996362 VOTES  - block total:  17278.000000000004 , precinct total:  17284.0
GSL006DMIL: DIFFERENCE OF -2.000000000003638 VOTES  - block total:  15428.999999999996 , precinct total:  15431.0
GSL006RWHA: DIFFERENCE OF -

IndexError: single positional indexer is out-of-bounds

In [ ]:
# Merge all election results gdfs back together
b_merge = st_bprec_votes.merge(cong_bprec_votes, how='outer', on=['GEOID20', 'NAME20','COUNTYFP20', 'PRECINCTID', 'VAP_MOD',"geometry"]).merge(
    sl_bprec_votes, how='outer', on=['GEOID20', 'NAME20','COUNTYFP20', 'PRECINCTID', 'VAP_MOD',"geometry"]).merge(
    su_bprec_votes, how='outer', on=['GEOID20', 'NAME20','COUNTYFP20', 'PRECINCTID', 'VAP_MOD',"geometry"])